In [25]:
# bf.get_boot_attrs(connection)

def bytes_to_hex_string(byte_data):
    hex_bytes = [f"{byte:02x}" for byte in bytes(byte_data)]
    return ' '.join(hex_bytes)


In [66]:
from _types import *

# la première commande est : # bf.get_boot_attrs(connection)
# qui fait : 
    # mem_range_response = _send_and_receive(
    #     connection,
    #     Command(CommandCode.GET_MEMORY_ADDRESS_RANGE),
    # )

# Checksum(42).__bytes__()

# len(bytes(Command(CommandCode.GET_MEMORY_ADDRESS_RANGE)))

# # ok, la commande pour demander fait 11 octets, et commence juste par "0b"
# # ensuite, déparser la réponse
v = Version(ResponseCode.SUCCESS)
v.version = 42
v.max_packet_length = 43
v.device_id = 45
v.erase_size = 48
v.write_size = 34

bytes_to_hex_string(v.__bytes__())


'01 00 00 00 00 00 00 00 00 00 00 2a 00 2b 00 00 00 2d 00 00 00 30 00 22 00 00 00 00 00 00 00 00 00 00 00 00 00'

In [91]:
from _util import chunked

bootattrs = BootAttrs(
        version=258,
        max_packet_length=256,
        device_id=13398,
        erase_size=2048,
        write_size=8,
        memory_range=(6144, 174080),
        has_checksum=True,
    )
total_bytes, chunks = chunked("../mcbootflash/tests/testcases/flash/test.hex", bootattrs)

chunks_array = []
for chunk in chunks:
    print("CHUNK : ")
    print(chunk.address)
    print(bytes_to_hex_string(chunk.data))
    print("        ")

CHUNK : 
6144
e0 1a 04 00 00 00 00 00 02 00 fa 00 00 0f 78 00 1e 00 78 00 00 40 78 00 67 40 60 00 00 80 fb 00 67 00 60 00 4a 00 dd 00 02 0a 80 00 f1 3f 2e 00 81 00 61 00 01 00 70 00 00 0a 88 00 00 80 fa 00 00 00 06 00 02 00 fa 00 00 0f 78 00 1e 00 78 00 00 40 78 00 67 40 60 00 00 80 fb 00 67 00 60 00 02 0a 80 00 81 ff 2f 00 81 00 61 00 01 00 70 00 00 0a 88 00 00 80 fa 00 00 00 06 00 00 00 fa 00 43 01 a8 00 00 80 fa 00 00 00 06 00 00 00 fa 00 00 28 a9 00 00 80 fa 00 00 00 06 00 02 00 fa 00 00 0f 78 00 1e 00 78 00 00 40 78 00 67 40 60 00 00 80 fb 00 67 00 60 00 4a 00 dd 00 42 0a 80 00 f1 3f 2e 00 81 00 61 00 01 00 70 00 40 0a 88 00 00 80 fa 00 00 00 06 00 02 00 fa 00 00 0f 78 00 1e 00 78 00 00 40 78 00 67 40 60 00 00 80 fb 00
        
CHUNK : 
6264
67 00 60 00 42 0a 80 00 81 ff 2f 00 81 00 61 00 01 00 70 00 40 0a 88 00 00 80 fa 00 00 00 06 00 00 00 fa 00 4b 01 a8 00 00 80 fa 00 00 00 06 00 02 00 fa 00 00 0f 78 00 1e 00 78 00 00 40 78 00 67 40 60 00 00 80 fb 00 67 00 60 00 4a 00 dd 00 82 

In [ ]:
# Connect to a device in bootloader mode.
connection = serial.Serial(port=<PORT>, baudrate=<BAUDRATE>, timeout=<TIMEOUT>)
# Query its attributes.
bootattrs = bf.get_boot_attrs(connection)
# Load the firmware image and split it into chunks.
total_bytes, chunks = bf.chunked(hexfile=<HEXFILE_PATH_STRING>, bootattrs)
# Erase the device's program memory area.
bf.erase_flash(connection, bootattrs.memory_range, bootattrs.erase_size)

# Write the firmware chunks to the bootloader in a loop.
for chunk in chunks:
    bf.write_flash(connection, chunk)

    # Optionally, check that the write is OK by checksumming.
    if bootattrs.has_checksum:
        bf.checksum(connection, chunk)

    # At this point, you may want to give an indication of the flashing progress,
    # like updating a progress bar.

# Verify that the new application is detected.
bf.self_verify(connection)


In [98]:
# test du crc ! 
import binascii
def crc_srec(hexstr):
    """Calculate the CRC for given Motorola S-Record hexstring.

    """

    crc = sum(binascii.unhexlify(hexstr))
    crc &= 0xff
    crc ^= 0xff

    return crc

#    std::vector<uint8_t> bytes{67, 56, 89, 45};
print(crc_srec("4338592d") == 254)


crc_srec("588405bc7be1f154bbab51427e254050425dcf55") == 210

True


True

les étapes du code : 

get_boot_attrs : 

# demander VERSION
    read_version_response = _send_and_receive(
        connection,
        Command(CommandCode.READ_VERSION),
    )

# demander MEMORY_RANGE
    mem_range_response = _send_and_receive(
        connection,
        Command(CommandCode.GET_MEMORY_ADDRESS_RANGE),
    )

    assert isinstance(mem_range_response, MemoryRange)

    # program_end + 2 explanation:
    # +1 because the upper bound reported by the bootloader is inclusive, but we
    # want to use it as a Python range, which is half-open.
    # +1 because the final byte of the final 24-bit instruction is not included in
    # the range reported by the bootloader, but it is still writable.
    return mem_range_response.program_start, mem_range_response.program_end + 2


# demander CHECKSUM (peut planter)
    try:
        _get_remote_checksum(connection, memory_range[0], write_size)
        has_checksum = True
    except UnsupportedCommand:
        _logger.warning("Bootloader does not support checksumming")
        has_checksum = False

    checksum_response = _send_and_receive(
        connection,
        Command(
            command=CommandCode.CALC_CHECKSUM,
            data_length=length,
            address=address,
        ),
    )
    assert isinstance(checksum_response, Checksum)

# on a tous les boot_attrs : 
    return BootAttrs(
        version,
        max_packet_length,
        device_id,
        erase_size,
        write_size,
        memory_range,
        has_checksum,
    )


# ensuite, on chunke tout : 

total_bytes, chunks = bf.chunked(hexfile=<HEXFILE_PATH_STRING>, bootattrs)


# on erase la flash : 
    _send_and_receive(
        connection,
        command=Command(
            command=CommandCode.ERASE_FLASH,
            data_length=(end - start) // erase_size,
            unlock_sequence=_FLASH_UNLOCK_KEY,
            address=start,
        ),
    )


# pour tous les chunks, on fait : 

    _send_and_receive(
        connection,
        Command(
            command=CommandCode.WRITE_FLASH,
            data_length=len(chunk.data),
            unlock_sequence=_FLASH_UNLOCK_KEY,
            address=chunk.address,
        ),
        chunk.data,
    )






In [ ]:
# ok, il faut tester l'opération de chunkage maintenant, le reste y'a pas grand chose : 



